<a href="https://colab.research.google.com/github/sachin0034/MLAI-community-labs/blob/sachin-001/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1 - Install required packages
!pip install crewai pyboxen langchain python-dotenv PyPDF2 agentops openai

In [ ]:
# Cell 2 - Import libraries and mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
from dotenv import load_dotenv
import os
from crewai import Crew, Agent
from pyboxen import boxen
import agentops
from langchain.chat_models import ChatOpenAI
# Initialize environment variables
load_dotenv()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


False

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving .env to .env
User uploaded file ".env" with length 292 bytes


In [ ]:
# Cell 3 - Initialize AgentOps and API key
agentops.init(tags=['contract-analysis'])
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

🖇 AgentOps: The 'tags' parameter is deprecated. Use 'default_tags' instead
🖇 AgentOps: Could not initialize AgentOps client - API Key is missing.
	    Find your API key at https://app.agentops.ai/settings/projects


In [ ]:
# Cell 4 - Define reference data
reference_data = {
    "limitation of liability": "30 days",
    "owner expiry date": "30 days",
    "notice period": "30 days",
    "agreement duration": "12 months",
    "payment terms": "30 days",
    "confidentiality period": "3 years",
    "insurance coverage": "$1,000,000",
    "maximum monthly hours": "120"
}

print(boxen(
    "\n".join([f"{k}: {v}" for k, v in reference_data.items()]),
    title="📋 Reference Contract Terms",
    padding=1,
    margin=1,
    color="blue"
))

                                                                                                                   
   ╭─ 📋 Reference Contract Terms ────────╮                                                                        
   │                                      │                                                                        
   │   limitation of liability: 30 days   │                                                                        
   │   owner expiry date: 30 days         │                                                                        
   │   notice period: 30 days             │                                                                        
   │   agreement duration: 12 months      │                                                                        
   │   payment terms: 30 days             │                                                                        
   │   confidentiality period: 3 years    │                              

In [ ]:
from langchain.tools import tool
from PyPDF2 import PdfReader
from typing import Optional

@tool
def pdf_reader(file_path: str) -> str:
    """Read and extract text from PDF files"""
    try:
        with open(file_path, 'rb') as file:
            reader = PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
        return text
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def reference_data_tool(query: Optional[str] = None) -> dict:
    """Access the standard reference data for comparison"""
    return reference_data

In [ ]:
# Cell 6 - Create memory system tables
from rich import box
from rich.console import Console
from rich.table import Table
from rich.syntax import Syntax

def create_tables():
    console = Console()

    components_table = Table(box=box.DOUBLE, title="Memory System Components",
                           show_header=True, header_style="bold cyan", show_lines=True)

    components_table.add_column("Component", style="bold white")
    components_table.add_column("Description", style="dim white")

    components = [
        ("Short-Term Memory", "Temporarily stores recent interactions and outcomes using RAG, enabling agents to recall and utilize information relevant to their current context during executions."),
        ("Long-Term Memory", "Preserves valuable insights and learnings from past executions, allowing agents to build and refine their knowledge over time."),
        ("Entity Memory", "Captures and organizes information about entities (people, places, concepts) encountered during tasks, facilitating deeper understanding and relationship mapping. Uses RAG for storing entity information."),
        ("Contextual Memory", "Maintains the context of interactions by combining ShortTermMemory, LongTermMemory, and EntityMemory, aiding in coherence and relevance of agent responses."),
        ("User Memory", "Stores user-specific information and preferences, enhancing personalization and user experience.")
    ]

    for component, desc in components:
        components_table.add_row(component, desc)

    console.print(components_table)

if __name__ == "__main__":
    create_tables()

                                             Memory System Components                                              
╔═══════════════════╦═════════════════════════════════════════════════════════════════════════════════════════════╗
║ Component         ║ Description                                                                                 ║
╠═══════════════════╬═════════════════════════════════════════════════════════════════════════════════════════════╣
║ Short-Term Memory ║ Temporarily stores recent interactions and outcomes using RAG, enabling agents to recall    ║
║                   ║ and utilize information relevant to their current context during executions.                ║
╠═══════════════════╬═════════════════════════════════════════════════════════════════════════════════════════════╣
║ Long-Term Memory  ║ Preserves valuable insights and learnings from past executions, allowing agents to build    ║
║                   ║ and refine their knowledge over time.                                                       ║
╠═══════════════════╬═════════════════════════════════════════════════════════════════════════════════════════════╣
║ Entity Memory     ║ Captures and organizes information about entities (people, places, concepts) encountered    ║
║                   ║ during tasks, facilitating deeper understanding and relationship mapping. Uses RAG for      ║
║                   ║ storing entity information.                                                                 ║
╠═══════════════════╬═════════════════════════════════════════════════════════════════════════════════════════════╣
║ Contextual Memory ║ Maintains the context of interactions by combining ShortTermMemory, LongTermMemory, and     ║
║                   ║ EntityMemory, aiding in coherence and relevance of agent responses.                         ║
╠═══════════════════╬═════════════════════════════════════════════════════════════════════════════════════════════╣
║ User Memory       ║ Stores user-specific information and preferences, enhancing personalization and user        ║
║                   ║ experience.                                                                                 ║
╚═══════════════════╩═════════════════════════════════════════════════════════════════════════════════════════════╝

In [ ]:
from langchain_openai import ChatOpenAI

def create_agents():
    researcher_agent = Agent(
        role='PDF Data Extractor',
        goal='Extract key contract terms from PDF',
        backstory='Expert in extracting contract terms from PDFs',
        llm=ChatOpenAI(temperature=0.7, model='gpt-4'),
        verbose=True,
        tools=[pdf_reader],
        memory=True,
    )

    writer_agent = Agent(
        role='Data Comparison Analyst',
        goal='Compare contract terms with reference values',
        backstory='Expert in analyzing contract terms',
        llm=ChatOpenAI(temperature=0.7, model='gpt-4'),
        verbose=True,
        tools=[reference_data_tool],
        memory=True,
    )

    review_agent = Agent(
        role='Compliance Report Generator',
        goal='Create detailed report of contract deviations',
        backstory='Expert in compliance reporting',
        llm=ChatOpenAI(temperature=0.7, model='gpt-4'),
        verbose=True,
        tools=[reference_data_tool],
        memory=True,
    )

    for agent in [researcher_agent, writer_agent, review_agent]:
        print(boxen(
            f"Role: {agent.role}\nGoal: {agent.goal}\nBackstory: {agent.backstory}",
            title="🤖 Agent Profile",
            padding=1,
            margin=1,
            color="green"
        ))

    return researcher_agent, writer_agent, review_agent

# Initialize agents
researcher_agent, writer_agent, review_agent = create_agents()

                                                                                                                   
   ╭─ 🤖 Agent Profile ───────────────────────────────────────────╮                                                
   │                                                              │                                                
   │   Role: PDF Data Extractor                                   │                                                
   │   Goal: Extract key contract terms from PDF                  │                                                
   │   Backstory: Expert in extracting contract terms from PDFs   │                                                
   │                                                              │                                                
   ╰──────────────────────────────────────────────────────────────╯                                                
                                                                         

                                                                                                                   
   ╭─ 🤖 Agent Profile ─────────────────────────────────────╮                                                      
   │                                                        │                                                      
   │   Role: Data Comparison Analyst                        │                                                      
   │   Goal: Compare contract terms with reference values   │                                                      
   │   Backstory: Expert in analyzing contract terms        │                                                      
   │                                                        │                                                      
   ╰────────────────────────────────────────────────────────╯                                                      
                                                                         

                                                                                                                   
   ╭─ 🤖 Agent Profile ──────────────────────────────────────╮                                                     
   │                                                         │                                                     
   │   Role: Compliance Report Generator                     │                                                     
   │   Goal: Create detailed report of contract deviations   │                                                     
   │   Backstory: Expert in compliance reporting             │                                                     
   │                                                         │                                                     
   ╰─────────────────────────────────────────────────────────╯                                                     
                                                                         

In [ ]:
# Cell 8 - Create tasks
from crewai import Task
from textwrap import dedent

def create_tasks(researcher_agent, writer_agent, review_agent, input_file):
    pdf_extraction_task = Task(
        description=dedent(f"""Use the PDF Reader tool to read {input_file} and extract key terms."""),
        agent=researcher_agent,
        expected_output="Dictionary containing extracted contract terms"
    )

    comparison_task = Task(
        description=dedent(f"""Compare extracted terms with reference data and identify mismatches."""),
        agent=writer_agent,
        expected_output="List of mismatches between extracted and reference data"
    )

    final_report_task = Task(
        description=dedent(f"""Create comprehensive analysis report with findings."""),
        agent=review_agent,
        expected_output="Detailed analysis report with findings and recommendations"
    )

    for task in [pdf_extraction_task, comparison_task, final_report_task]:
        print(boxen(
            f"Description: {task.description}\nExpected Output: {task.expected_output}",
            title="📝 Task Details",
            padding=1,
            margin=1,
            color="yellow"
        ))

    return pdf_extraction_task, comparison_task, final_report_task

In [ ]:
# Cell 9 - Run analysis
# Specify the path to your PDF file in Google Drive
input_file = "/content/drive/MyDrive/contract.pdf"
tasks = create_tasks(researcher_agent, writer_agent, review_agent, input_file)

def run_analysis(agents, tasks):
    # Remove any leading/trailing whitespaces or newline characters from the API key
    os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY").strip()

    crew = Crew(
        agents=agents,
        tasks=tasks,
        verbose=True
    )

    result = crew.kickoff()

    # Convert the CrewOutput object to a string before passing it to boxen
    result_str = str(result)  # or result.raw if it has a 'raw' attribute

    print(boxen(
        result_str,  # Pass the string representation to boxen
        title="📊 Contract Analysis Report",
        padding=1,
        margin=1,
        color="magenta"
    ))

    return result

# Execute analysis
final_report = run_analysis([researcher_agent, writer_agent, review_agent], tasks)

                                                                                                                   
   ╭─ 📝 Task Details ─────────────────────────────────────────────────────────────────────────────────────────╮   
   │                                                                                                           │   
   │     Description: Use the PDF Reader tool to read /content/drive/MyDrive/contract.pdf and extract key      │   
   │     terms.                                                                                                │   
   │     Expected Output: Dictionary containing extracted contract terms                                       │   
   │                                                                                                           │   
   ╰───────────────────────────────────────────────────────────────────────────────────────────────────────────╯   
                                                                         

                                                                                                                   
   ╭─ 📝 Task Details ─────────────────────────────────────────────────────────────────────╮                       
   │                                                                                       │                       
   │   Description: Compare extracted terms with reference data and identify mismatches.   │                       
   │   Expected Output: List of mismatches between extracted and reference data            │                       
   │                                                                                       │                       
   ╰───────────────────────────────────────────────────────────────────────────────────────╯                       
                                                                                                                   



                                                                                                                   
   ╭─ 📝 Task Details ───────────────────────────────────────────────────────────────╮                             
   │                                                                                 │                             
   │   Description: Create comprehensive analysis report with findings.              │                             
   │   Expected Output: Detailed analysis report with findings and recommendations   │                             
   │                                                                                 │                             
   ╰─────────────────────────────────────────────────────────────────────────────────╯                             
                                                                                                                   

# Agent: PDF Data Extractor
## Task: Use the PDF Reader tool to read /co

                                                                                                                   
   ╭─ 📊 Contract Analysis Report ─────────────────────────────────────────────────────────────────────────────╮   
   │                                                                                                           │   
   │   Detailed Analysis Report:                                                                               │   
   │                                                                                                           │   
   │   1. Agreement Duration: No deviation found. The extracted data matches the reference data.               │   
   │                                                                                                           │   
   │   2. Payment terms: Deviation found. Extracted data is "Net 45 days" whereas the reference data is "30    │   
   │   days". Recommendation: Revise the payment terms to align with the 